In [ ]:
#schema for classic cars - bc it's a transactional databases it's normalized
# get every piece of info about an order by the order id, info in order, get customer name, get order items
# product code goes to product table
# JOIN

# 3.1

In [ ]:

#Returning JSON Data
def get_order(order_no):
    q = """
    SELECT
    orders.orderNumber as order_number, orders.orderData as order_date,
    (select customers.customerNumber as
        customer_number from customers where 
        customers.customerNumber=orders.customerNumber) as customer_number,
    (select customers.customerName as customer_name from
        customers where customers.customerNumber=orders.customerNumber)
        as customer_name,
    orderdetails.productCode as product_code,
    (select productName as product_name from products where 
        products.productCode = orderdetails.productCode) as product_name,
    orderdetails.quantityOrdered as quantity,
    orderdetails.priceEach as unit_price,
    orderdetails.orderLineNumber as line_number
    FROM
    orders
    JOIN 
    orderdetails
    on
    orders.orderNumber = orderdetails.orderNumber
    where
    orders.orderNumber = %s
    order by orderLineNumber
    """
    
    cnx = get_new_connection()
    result = run_q(cnx,q,(int(order_no)),fetch=True, commit=True)
    
    if len(result)>=0:
        final_result = {}
        final_result['customer_name'] = result[0]['customer_name']
        final_result['customer_number'] = result[0]['customer_number']
        final_result['order_number'] = result[0]['order_number']
        final_result['order_date'] = result[0]['order_date']
        
    final_result['order_items'] = []
    for r in result:
        new item = {}
        new_item['No.'] = r['line_number']
        new_item['product_code'] = r['product_code']
        new_item['product_name'] = r['product_name']
        new_item['quantity'] = r['quantity']
        new_item['unit_price'] = float(r['unit_price'])
        final_result['order_items'].append(new_item)
        
    return final_result

    #return result?????

        
                                                
    
    
    
   



# 3.2

In [ ]:
q = """
select c.*,
    (select nameLast from lahman2017.people where
        lahman2017.people.playerID=c.playerID) as last_name,
    (select nameFirst from lahman2017.people where
        lahman2017.people.playerID=c.playerID) as first_name,
    d.avg_salary, d.salary_seasons, c.batting_average/d.avg_salary*1000 as ba_dollar from
    (select a.*, b.total_hits, b.total_abs, b.batting_average from
    (
        SELECT playerID, sum(InnOuts) total_outs, sum(gs) total_gs,
        sum(g) as games, round(sum(InnOuts)/27) as computed_games FROM lahman2017.Fielding
        where pos = 'C' and yearID >= 1950
        group by playerID
        having computer_games > 162
    ) as a 
    join
        (SELECT playerID, sum(h) as total_hits, sum(ab) as total_abs, round(sum(h))/sum(ab),3)
        as batting_average from lahman2017.Batting
        where yearID >= 1950 and ab > 0 group by playerID) as b
    on
        a.playerID = b.playerID) as c
        
    join
        (select playerID, round(avg(salary)) as avg_salary, count(*) as salary_seasons from
        lahman2017.salaries group by playerID having salary_seasons >= 3 and avg_salary > 0) as d
    on
        c.playerID = d.playerID
    order by batting_average desc
"""

new_c = get_new_connection()
df = pd.read_sql(q,new_c)


In [ ]:
df.head(10)

In [ ]:
#some subsequent analysis for funsies maybe
df2 = df[['playerID','last_name','first_name','avg_salary','batting_average']].copy()
df2['ba_per_dollar']=df2['batting_average']/df2['avg_salary'] * 100000
df3 = df2.sort_values(by=['ba_per_dollar'], ascending=False)
df3.head[10]

# 3.3

In [ ]:
#building it up using a series of left-joins
#compute aggregate performances join them and produce the answer
#full outer join- left join and right join and them union them
# extract from the sql query he gave



# 3.4

In [ ]:
#edit schema defintiions

# 3.5

In [1]:
#define sql schema and draw simple diagram
# write 3 functions

In [ ]:
#Diagram and create table statements - check pictures

In [ ]:
#Function to create a uni

#Make a faculty

def compute_uni_prefix(last_name,first_name):
    result = last_name[0:2] + first_name[0:2]
    return result

def create_faculty(last_name, first_name, email, title):
    try: 
        cnx = get_new_connection()
        cur = cnx.cursor()
        cur.execute("SET SESSION TRANSACTION ISOLATION LEVEL SERIALIZABLE")
        
        prefix = compute_uni_prefix(last_name,first_name)
        prefix = prefix.lower()
        
        q1 = 'select count(*) as count from w4111final.person where uni like(%s)''
        result = run_q(cnx,q1,prefix+"%", True, False)
        
        count = result[0]['count'] + 1
        new_uni = prefix + str(count)
        
        q2 = "insert into w4111final.person values(%s, %s, %s, %s)"
        result = run_q(cnx,q2,(new_uni, last_name, first_name, email), fetch=False, commit=False)
        
        
        q3 = "insert into w4111final.faculty values(%s, %s)"
        result = run_q(cnx, q3, (new_uni, title), fetch = False, commit=False)
        
        cnx.commit()
        cnx.close()
    except Exception as e:
        print("Exception = ", e)
        cnx.rollback()
        cnx.close()
        
def create_student(last_name, first_name, email, major, advisor_uni):
     try: 
        cnx = get_new_connection()
        cur = cnx.cursor()
        cur.execute("SET SESSION TRANSACTION ISOLATION LEVEL SERIALIZABLE")
        
        prefix = compute_uni_prefix(last_name,first_name)
        prefix = prefix.lower()
        
        q1 = 'select count(*) as count from w4111final.person where uni like(%s)''
        result = run_q(cnx,q1,prefix+"%", True, False)
        
        count = result[0]['count'] + 1
        new_uni = prefix + str(count)
        
        q2 = "insert into w4111final.person values(%s, %s, %s, %s)"
        result = run_q(cnx, q2, (new_uni, last_name, first_name, email), fetch = False, commit=False)
        
        q3 = "insert into w4111final.student (uni, major, advisor) values(%s, %s, %s)"
        result = run_q(cnx, q3, (new_uni, major, advisor_uni), fetch = False, commit=False)
        
        cnx.commit()
        cnx.close()
    except Exception as e:
        print("Exception = ", e)
        cnx.rollback()
        cnx.close()
        
def retrieve_person(uni):
    try: 
        cnx = get_new_connection()
        cur = cnx.cursor()
        
        q1 = "select * from w4111final.person where uni=%s"
        result = run_q(cnx,q1,uni,True,True)
        
        result = result[0]
        cnx.close()
        
        return result
    
    except Exception as e:
        print("Exception = ", e)
        cnx.close()
        
def retrieve_student(uni):
    try: 
        cnx = get_new_connection()
        cur = cnx.cursor()
        
        q1 = "select w4111final.person.*, w4111final.student.major, " + \
        " w4111final.student.advisor from w4111final.person join w4111final.student " + \
        "on w4111final.student.uni=w4111final.person.uni where w4111final.person.uni=%s"
        result = run_q(cnx,q1,uni,True,True)
        
        result = result[0]
        
        cnx.close()
        return result
    
    except Exception as e:
        cnx.close()
        print("Exception = ", e)
        

# Tests    
        

In [ ]:
#idk where this goes

In [ ]:
def create_faculty(self, last_name, first_name, email, title):
    uni_prefix = compute_unit_prefix(last_name, first_name)
    uni_prefix = unit_prefix.lower()
    
    tx = self.__graph.begin(autocommit=False)
    
    try:
        q1 = "match (p:Person) where p.uni starts with {x} return count (p) as count"
        cnt = self.run_q(q1,x=uni_prefix)
        cnt = cnt[0]['count'] + 1
        
        uni = uni_prefix + str(cnt)
        
        n = Node("Person", "Faculty", uni=uni, last_name=last_name, first_name=first_name, email=email, title=title)
        
        tx.create(n)
        tx.commit()
        return uni
    except Exception as e:
        print("Exception = ", e)
        tx.rollback()
        
def get_faculty(self, uni):
    n = self.find_nodes_by_template({"label": "Faculty", "template": {"uni": uni}})
    if n is not None and len(n) > 0:
        n = n[0]
    else:
        n = None
    return n

def create_student(self, last_name, first_name, email, major, advisor):
    uni_prefix = compute_uni_prefix(last_name,first_name)
    uni_prefix = uni_prefix.lower()
    
    tx = self._graph.begin(autocommit=False)
    
    try: 
        q1 = "match (p:Person) where p.uni starts with {x} return count(p) as count"
        cnt = self.run_q(q1, x=uni_prefix)
        cnt = cnt[0]['count'] + 1 
        uni = uni_prefix + str(cnt)

        
        f = self.get_facultry(advisor)
        if f is None:
            raise ValueError("Advisor does not exist!")
            
        n = Node("Person", "Student", uni=uni, last_name=last_name, first_name=first_name,major=major, email=email)
        tx.create(n)
        
        r = Relationship(f, "ADVISES", n)
        tx.create(r)
        
        tx.commit()
        
        return uni
    
    except Exception as e:
        print("Exception = ", e)
        tx.rollback()
        raise(e)
        
def get_person(self, uni):
    person_fields = ['uni', 'last_name', 'first_name', 'email']
    
    n = self.find_nodes_by_template({"label":"Person", "template" : {"uni":uni}})

    
    if n is not None and len(n) >0:
        n = n[0]
    else:
        n=None
        return n
    key = copy.copy(list(n.keys()))
    
    for k in keys:
        if k not in person_fieds:
            del(n[k])
        return n
    
def get_student(self, uni):
    
    n = self.find_nodes_by_template({"label":"Person", "template" : {"uni":uni}})
    
    if n is not None and len(n)>0:
        n = n[0]
    else:
        n = None
        return n
    return n
        

    

In [ ]:
#redis?


In [ ]:
#def add_to_cache
#def get_from_cache


def get_uni(last_name, first_name):
    uni_prefix = last_name[0:2] + first_name[0:2]
    uni_prefix = uni_prefix.lower()
    
    key_template = "Person:*:" + uni_prefix + "*"
    result = r.keys(key_template)
    next_num = len(result) + 1
    
    result = uni_prefix + str(next_num)
    
    return result

def create_faculty(last_name, first_name, email, title):
    new_uni = get_uni(last_name, first_name)
    new_key = "Person:Faculty:" + new_uni
    props = {
        "last_name" : last_name,
        "first_name" : first_name,
        "email" : email,
        "title": title,
        "uni" : new_uni
    }
    add_to_cache(new_key,props)
    
    return new_uni

def create_student(last_name, first_name, email, major, advisor):
    new_uni = get_uni(last_name, first_name)
    new_key = "Person:Student:" + new_uni
    props = {
        "last_name" : last_name,
        "first_name" : first_name,
        "email" : email,
        "major": major,
        "uni" : new_uni
    }
    add_to_cache(new_key,props)
    r.lpush("Advises:" + advisor, new_uni)
    
    
    return new_uni


def get_student(uni):
    new_uni = "Person:Student:" + uni
    result = get_from_cache(new_uni)
    return result

def get_person(uni):
    person_fields = ['last_name', 'first_name', 'email','uni']
    
    pattern = "Person(*)" + uni
    keys = r.keys(pattern)
    
    result = None
    
    if keys is not None and len(keys) == 1:
        k = keys[0]
        tmp = get_from_cache(k)
        if tmp is not None and len(tmp) > 0:
            result = tmp
            
        if k is not None:
            keys=copy.copy(list(result.keys()))
            for k in keys:
                if k is not in person_fields:
                    del(result[k])
        return result
    
def get_student(uni):
    key = "Person:Student:" + uni
    return get_from_cache(key)
    return result

def get_advisees(uni):
    key = "Advises:" + uni
    result = r.lrange(key, 0, -1)
    return result
    
    
    